In [2]:
import zipfile
import geopandas as gpd
import requests

zip_file_path = 'tl_2020_24_tabblock20.zip'

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files to the current working directory
    zip_ref.extractall()

shapefile_path = 'tl_2020_24_tabblock20.shp'
block_gdf = gpd.read_file(shapefile_path)

In [3]:
url = "https://transit.land/api/v2/rest/feed_versions/8523f088e05dcd273b369dd4d65eb771c73c0a22/download?apikey=T2Jljq0EQqqe1JMfijkn3RdtVxoYpwuq"
response = requests.get(url)

if response.status_code == 200:
    with open("output.zip", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download the file.")

File downloaded successfully.


In [3]:
import os

zip_file_path = 'output.zip'
extract_to_folder = 'transitland'
os.makedirs(extract_to_folder, exist_ok=True)

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files to the current working directory
    zip_ref.extractall(extract_to_folder)

In [4]:
import pandas as pd

In [5]:
#CODE FOR BLOCKS
compressed_csv_file_home = 'md_rac_S000_JT00_2020.csv.gz'
compressed_csv_file_work = 'md_wac_S000_JT00_2020.csv.gz'
gdf_filtered = block_gdf[block_gdf['COUNTYFP20'] == '510']

df_origins_blocks = pd.read_csv(compressed_csv_file_home)
df_destinations_blocks = pd.read_csv(compressed_csv_file_work)

df_origins_blocks['h_geocode'] = df_origins_blocks['h_geocode'].astype(str)
df_destinations_blocks['w_geocode'] = df_destinations_blocks['w_geocode'].astype(str)

df_origins_blocks = df_origins_blocks[(df_origins_blocks['h_geocode'].str[:5] == '24510') & (df_origins_blocks['C000'] != 0)]
df_destinations_blocks = df_destinations_blocks[(df_destinations_blocks['w_geocode'].str[:5] == '24510') & (df_destinations_blocks['C000'] != 0)]

df_origins_blocks['h_geocode'] = df_origins_blocks['h_geocode'].astype(int)
df_destinations_blocks['w_geocode'] = df_destinations_blocks['w_geocode'].astype(int)

gdf_filtered['GEOID20'] = gdf_filtered['GEOID20'].astype(int)
origins_merged_blocks = gdf_filtered.merge(df_origins_blocks, left_on = 'GEOID20', right_on = 'h_geocode')
destinations_merged_blocks = gdf_filtered.merge(df_destinations_blocks, left_on = 'GEOID20', right_on = 'w_geocode')

#origins_merged = origins_merged[['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry', 'h_geocode', 'C000']]
#destinations_merged = destinations_merged[['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry', 'w_geocode', 'C000']]

origins_merged_blocks['id'] = origins_merged_blocks.index
destinations_merged_blocks['id'] = destinations_merged_blocks.index

/Users/yangxinyuxie/miniforge3/envs/aaforecast/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
from shapely.geometry import Point

# Convert latitude and longitude columns to float data types
origins_merged_blocks['INTPTLAT20'] = origins_merged_blocks['INTPTLAT20'].astype(float)
origins_merged_blocks['INTPTLON20'] = origins_merged_blocks['INTPTLON20'].astype(float)
origins_merged_blocks['geometry'] = origins_merged_blocks.apply(lambda row: Point(row['INTPTLON20'], row['INTPTLAT20']), axis=1)

destinations_merged_blocks['INTPTLAT20'] = destinations_merged_blocks['INTPTLAT20'].astype(float)
destinations_merged_blocks['INTPTLON20'] = destinations_merged_blocks['INTPTLON20'].astype(float)
destinations_merged_blocks['geometry'] = destinations_merged_blocks.apply(lambda row: Point(row['INTPTLON20'], row['INTPTLAT20']), axis=1)

In [7]:
accurate_times = pd.read_csv('bus_accurate_data.csv')
accurate_times['scheduled_visit_time'] = pd.to_datetime(accurate_times['scheduled_visit_time'])
accurate_times['date'] = accurate_times['scheduled_visit_time'].dt.date
accurate_times.rename(columns={'vehicle_stop_id': 'stop_id'}, inplace=True)
def filter_by_time(trip_data):
    hour = trip_data.reset_index()['scheduled_visit_time'][0].hour
    if (hour >= 7) and (hour < 10):
        return trip_data
    else:
        return None

accurate_times = accurate_times.groupby("trip_id").apply(filter_by_time).reset_index(drop=True)
accurate_times['scheduled_visit_time'] = accurate_times['scheduled_visit_time'].dt.strftime('%H:%M:%S')
observed_mask = accurate_times['observed_visit_time'].notna()
accurate_times.loc[observed_mask, ['observed_visit_time']] = pd.to_datetime(accurate_times['observed_visit_time'][observed_mask]).dt.strftime('%H:%M:%S')
unique_dates = accurate_times['date'].unique()
filtered_dfs = {}
for date in unique_dates:
    filtered_dfs[date] = accurate_times[accurate_times['date'] == date]

/Users/yangxinyuxie/miniforge3/envs/aaforecast/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
from datetime import datetime

gtfs_stop_times = pd.read_csv('transitland/stop_times.txt')

for date, filtered_df in filtered_dfs.items():
    merged_df = gtfs_stop_times.merge(filtered_df[['trip_id', 'stop_id', 'observed_visit_time', 
                                                   'scheduled_visit_time', 'trip_distance_traveled']], on=['trip_id', 'stop_id'], how="left")

    merged_df = merged_df[merged_df['scheduled_visit_time'].notna()]

    observed_mask = merged_df['observed_visit_time'].notna()
    merged_df.loc[observed_mask, ['arrival_time', 'departure_time']] = merged_df.loc[observed_mask, 'observed_visit_time']
    merged_df.loc[~observed_mask, ['arrival_time', 'departure_time']] = ''

    merged_df.drop(columns=['observed_visit_time'], inplace=True)

    merged_df.to_csv(f'stop_times/stop_times_{date}.txt', index=False)

In [9]:
# for date, filtered_df in filtered_dfs.items():
#     gtfs_stop_times = pd.read_csv('transitland/stop_times.txt')
#     gtfs_stop_times = gtfs_stop_times.merge(filtered_df[['trip_id', 'stop_id', 'observed_visit_time', 'scheduled_visit_time']], on=['trip_id', 'stop_id'], how = "left")

#     observed_mask = gtfs_stop_times['scheduled_visit_time'].notna()
#     gtfs_stop_times.loc[observed_mask, ['arrival_time', 'departure_time']] = gtfs_stop_times.loc[observed_mask, 'scheduled_visit_time']
#     gtfs_stop_times.loc[~observed_mask, ['arrival_time', 'departure_time']] = ''

#     gtfs_stop_times['arrival_time'] = pd.to_datetime(gtfs_stop_times['arrival_time'])
#     gtfs_stop_times['departure_time'] = pd.to_datetime(gtfs_stop_times['departure_time'])

#     gtfs_stop_times['arrival_time'] = gtfs_stop_times['arrival_time'].dt.strftime('%H:%M:%S')
#     gtfs_stop_times['departure_time'] = gtfs_stop_times['departure_time'].dt.strftime('%H:%M:%S')

#     gtfs_stop_times.drop(columns=['observed_visit_time'], inplace=True)

#     gtfs_stop_times.to_csv(f'transitland/stop_times_{date}.txt', index=False)

In [24]:
current_folder = os.getcwd()  # Get the current working directory (where your IPython Notebook is located)
stop_times_folder = os.path.join(current_folder, 'stop_times')

folder_path = stop_times_folder
file_paths = []

# Loop through the files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".txt"):
        file_paths.append(os.path.join(folder_path, file))

In [10]:
def to_time(time_string):
    return datetime.strptime(time_string, "%H:%M:%S")

def get_time_diff(before, after):
    return datetime.strptime(after, "%H:%M:%S") - datetime.strptime(before, "%H:%M:%S")

def get_time_diff_rev(after, before):
    return datetime.strptime(after, "%H:%M:%S") - datetime.strptime(before, "%H:%M:%S")

In [26]:
def fill_missing_times(trip_df):
    known_times = trip_df.dropna(subset=["arrival_time"])
    if (len(known_times) == len(trip_df)) or len(known_times) == 0:
        return trip_df
    
    # Iterate through known time points to calculate and apply estimated times
    for i in range(1, len(known_times)):
        prev_row = known_times.iloc[i - 1]
        next_row = known_times.iloc[i]
        mask = (trip_df["arrival_time"].isna()) & \
            (trip_df["trip_distance_traveled"] >= prev_row["trip_distance_traveled"]) & \
                (trip_df["trip_distance_traveled"] <= next_row["trip_distance_traveled"])
        if len(trip_df[mask]) > 0:
            # Calculate time difference and speed for this segment
            time_difference = get_time_diff(prev_row["arrival_time"], next_row["arrival_time"])
            distance_difference = next_row["trip_distance_traveled"] - prev_row["trip_distance_traveled"]
            speed = distance_difference / time_difference.total_seconds()
            
            # Calculate and update estimated arrival and departure times for missing rows within the segment
            trip_df.loc[mask, "arrival_time"] = to_time(prev_row["arrival_time"]) + \
                pd.to_timedelta((trip_df.loc[mask, "trip_distance_traveled"] - prev_row["trip_distance_traveled"]) / speed, unit="s")
            trip_df.loc[mask, "arrival_time"] = pd.to_datetime(trip_df['arrival_time'][mask]).dt.strftime('%H:%M:%S')
    
    known_times.reset_index(inplace= True)

    # Impute the first stop
    cur_row = known_times.iloc[0]
    mask = (trip_df["arrival_time"].isna()) & (trip_df["trip_distance_traveled"] <= cur_row["trip_distance_traveled"])
    if len(trip_df[mask]) > 0:
        
        trip_df.loc[mask, "arrival_time"] = \
            (to_time(cur_row['arrival_time']) - \
             trip_df["scheduled_visit_time"][mask].apply(get_time_diff, args = (cur_row["scheduled_visit_time"],))).dt.strftime('%H:%M:%S')

    # Impute the last stop
    cur_row = known_times.iloc[len(known_times) - 1]
    mask = (trip_df["arrival_time"].isna()) & (trip_df["trip_distance_traveled"] >= cur_row["trip_distance_traveled"])
    if len(trip_df[mask]) > 0:

        trip_df.loc[mask, "arrival_time"] = \
            (to_time(cur_row['arrival_time']) + \
             trip_df["scheduled_visit_time"][mask].apply(get_time_diff_rev, args = (cur_row["scheduled_visit_time"],))).dt.strftime('%H:%M:%S')
    

    trip_df['departure_time'] = trip_df['arrival_time']
    return trip_df

for file_path in file_paths:
    result_df = pd.read_csv(file_path)
    result_df = result_df.groupby("trip_id").apply(fill_missing_times)
    result_df.drop(columns=['scheduled_visit_time', 'trip_distance_traveled'], inplace=True)
    result_df.to_csv(file_path, index=False)  

In [27]:
import random
import os
import pandas as pd

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

def replace_data(trip_id):
    file_rand = random.sample(file_paths, len(file_paths))
    for filePath in file_rand:
        df = load_data(filePath)
        trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
        if trip_id not in trip_ids_to_replace:
            return df.loc[df.trip_id == trip_id] 

updated_folder = "stop_times_updated"
os.makedirs(updated_folder, exist_ok=True)  

for file in file_paths:
    df = load_data(file)
    trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
    for trip_id in trip_ids_to_replace:
        new_data = replace_data(trip_id)
        df = df.loc[df.trip_id != trip_id]
        df = df.append(new_data)
    trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
    #df.to_csv(f'updated_{file}', index=False)
    updated_file_path = os.path.join(updated_folder, f'updated_{os.path.basename(file)}')
    df.to_csv(updated_file_path, index=False)
    #df.to_csv(os.path.join(folder_path, f'{file}'), index=False)

In [29]:
import os
import shutil
import zipfile
from io import StringIO
import pandas as pd

stop_times_folder = 'stop_times_updated'
transitland_folder = 'transitland'
zipfiles_folder = 'zipfiles'
trips_folder = 'transitland/trips.txt'


for filename in os.listdir(stop_times_folder)[:1]:
    times = pd.read_csv(f"{stop_times_folder}/{filename}")
    trips = pd.read_csv('trips_copy.txt')  # Load trip_content as DataFrame
    unique_trip_ids = times['trip_id'].unique()
    trips = trips[trips['trip_id'].isin(unique_trip_ids)]
    
    #routes = pd.read_csv(f"{transitland_folder}/routes.txt")
    #unique_route_ids = trips['route_id'].unique()
    #routes = routes[routes['route_id'].isin(unique_route_ids)]
    #directions = pd.read_csv(f"{transitland_folder}/directions.txt")
    #directions = directions[directions['route_id'].isin(unique_route_ids)]
    trips.to_csv(trips_folder, index=False)
    times.to_csv(f'{transitland_folder}/stop_times.txt', index=False)
    #routes.to_csv(f"{transitland_folder}/routes.txt", index=False)
    #directions.to_csv(f"{transitland_folder}/directions.txt", index=False)

    
    zip_filename = f'{os.path.splitext(filename)[0]}.zip'
    zip_path = os.path.join(zipfiles_folder, zip_filename)
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(transitland_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, transitland_folder)
                zipf.write(file_path, arcname)
        

    print(f'Processed {filename}')


Processed updated_stop_times_2023-03-06.txt


In [30]:
import zipfile
import os

# Path to the zip file
zip_file_path = 'zipfiles/updated_stop_times_2023-03-06.zip'

# Directory where you want to extract the contents
extract_path = 'zipfiles_test'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extract_path)

print("Files extracted successfully!")


Files extracted successfully!


In [31]:
%%time
# Allow 6 GB memory
import sys
sys.argv.append(["--max-memory", "6G"])

import datetime
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransitMode, LegMode

# # Filepath to OSM data
osm_fp = "maryland-latest.osm.pbf"

CPU times: user 1.83 s, sys: 705 ms, total: 2.54 s
Wall time: 2.73 s


In [32]:
travel_time_matrix = []


# zip_path = os.path.join(zipfiles_folder, zip_filename)
transport_network = TransportNetwork(
    osm_fp,
    [
        zip_file_path,
    ]
)

In [33]:
travel_time_matrix_computer = TravelTimeMatrixComputer(
    transport_network,
    origins=origins_merged_blocks,
    destinations=destinations_merged_blocks,
    departure=datetime.datetime(2023,1,19,8,30),
    transport_modes=[TransitMode.BUS, LegMode.WALK],
)
travel_time_matrix_blocks = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix.append(travel_time_matrix_blocks)

In [41]:
travel_time_matrix[0].dropna()

,from_id,to_id,travel_time
7,0,7,37.0
8,0,8,101.0
10,0,10,57.0
25,0,25,30.0
26,0,26,22.0
...,...,...,...
3404,8291,3404,82.0
3405,8291,3405,106.0
3407,8291,3407,77.0
3408,8291,3408,62.0


In [17]:
import pandas as pd
import matplotlib.pyplot as plt

file_paths = ['transitland/stop_times_{}.txt'.format(date) for date in unique_dates]

data_frames = [pd.read_csv(file_path) for file_path in file_paths]
combined_df = pd.concat(data_frames, ignore_index=True)

missing_percentages = []

for trip_id in combined_df['trip_id'].unique():
    group = combined_df[combined_df['trip_id'] == trip_id]
    missing_percentage = (group['arrival_time'].isnull() & group['departure_time'].isnull()).mean() * 100
    missing_percentages.append(missing_percentage)
    if (missing_percentage > 80): 
        print(trip_id)

plt.hist(missing_percentages, bins=10, edgecolor='black')
plt.xlabel('Percentage of Missing Data')
plt.ylabel('Frequency')
plt.title('Histogram of Missing Data Percentage by Trip')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'transitland/stop_times_2023-01-19.txt'